In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/data_imputed_2.csv")

In [3]:
df.head()

,timestamp_local,temp,city_name,country_code,aqi,co,no2,o3,pm10,pm25,so2
0,01/02/2022 0:00,12.6,Gujrāt,PK,385,1339.8,76.0,10.7,491.7,347.67,238.0
1,01/02/2022 1:00,11.5,Gujrāt,PK,404,1437.6,76.0,9.3,508.3,359.33,268.0
2,01/02/2022 2:00,11.9,Gujrāt,PK,421,1535.5,76.0,8.0,525.0,371.00,298.0
3,01/02/2022 3:00,12.2,Gujrāt,PK,425,1659.0,68.3,5.3,529.3,374.00,275.7
4,01/02/2022 4:00,11.9,Gujrāt,PK,430,1782.5,60.7,2.7,533.7,377.00,253.3


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
temp = df[['temp']]

In [6]:
from sklearn.preprocessing import MinMaxScaler
values = temp.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [7]:
scaled = scaled.reshape((19706,))

In [8]:
scaled.shape

(19706,)

In [9]:
def df_to_X_y(df, window_size, forecast_length):
    # df_as_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df) - window_size - forecast_length):
        row_x = [[a] for a in df[i:i+window_size]]
        
        X.append(row_x)
        # row_y = df_as_np[i+window_size]
        row_y = [b for b in df[i+window_size:i+window_size+forecast_length]]
        
        y.append(row_y)
        
    return np.array(X), np.array(y)

In [10]:
WINDOW_WIDTH = 168
FORECAST_LENGTH = 168
X, y = df_to_X_y(scaled, WINDOW_WIDTH, FORECAST_LENGTH)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

X_max = X_train.max()
X_min = X_train.min()

In [11]:
from keras.models import Sequential, save_model, load_model
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

In [12]:
normalizer = Normalization()
normalizer.adapt(X_train)
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

model = Sequential()
# 1D Convolutional layers
model.add(Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# Flatten the output of the Conv layers
model.add(Flatten())
# model.add(LSTM(input_shape = X_train.shape[1:], units=86, return_sequences=True))
# model.add(LSTM(units=64, return_sequences=True))
# model.add(GRU(units=86, return_sequences=True))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(GRU(units=100, return_sequences=True))
# model.add(LSTM(units=100, return_sequences=False))
# model.add(Dense(units=100,  activation='relu'))
# model.add(Dense(8, activation='relu'))

model.add(Dense(units=FORECAST_LENGTH, activation='linear'))

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()] , loss=MeanSquaredError())

history = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[rlrop])

# score = model.evaluate(X_test, y_test, verbose = 1)

C:\Users\Hassan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 166, 8)              │              32 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1328)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 168)                 │         223,272 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 223,304 (872.28 KB)

 Trainable params: 223,304 (872.28 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0382 - root_mean_squared_error: 0.1766 - val_loss: 0.0050 - val_root_mean_squared_error: 0.0706 - learning_rate: 0.0010
Epoch 2/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0048 - root_mean_squared_error: 0.0694 - val_loss: 0.0047 - val_root_mean_squared_error: 0.0684 - learning_rate: 0.0010
Epoch 3/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0046 - root_mean_squared_error: 0.0680 - val_loss: 0.0046 - val_root_mean_squared_error: 0.0677 - learning_rate: 0.0010
Epoch 4/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0046 - root_mean_squared_error: 0.0675 - val_loss: 0.0044 - val_root_mean_squared_error: 0.0664 - learning_rate: 0.0010
Epoch 5/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0044 - root_mean_squared_error: 0.0660 - val_loss: 0.0043 - val_root_mean_squared_error: 0.0659 - learning_rate: 0.0010
Epoch 6/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0044 - root_mean_squared_err

In [13]:
forecast = model.predict(X_test)

122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [14]:
poll = np.array(df["temp"])

meanop = poll.mean()
stdop = poll.std()

y_test_true = y_test*stdop + meanop
testPredict = forecast*stdop + meanop

In [15]:
from sklearn.metrics import mean_squared_error

In [16]:
rmse = np.sqrt(mean_squared_error(y_test_true, testPredict))
print("Test RMSE ="  ,rmse)

Test RMSE = 0.559572


In [22]:
# Calculate MAPE
mape = np.mean(np.abs((y_test_true - testPredict) / y_test_true)) * 100
print("Test MAPE =", mape)

Test MAPE = 1.4858076348900795


In [24]:
# Calculate accuracy as 1 - MAPE
accuracy = 100 - mape
print("Accuracy =", accuracy, "%")

Accuracy = 98.51419236510992 %


In [27]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE
mae = mean_absolute_error(y_test_true, testPredict)
print("Test MAE =", mae)

Test MAE = 0.42024952


In [17]:
forecast

array([[0.54313886, 0.5302595 , 0.524672  , ..., 0.6047857 , 0.5936206 ,
        0.5767451 ],
       [0.86664593, 0.8551554 , 0.8362714 , ..., 0.7617025 , 0.7736183 ,
        0.8064643 ],
       [0.50036806, 0.56731975, 0.59275305, ..., 0.42572236, 0.46344888,
        0.4889831 ],
       ...,
       [0.4218175 , 0.40110525, 0.4004411 , ..., 0.27222347, 0.29774392,
        0.38311195],
       [0.58434635, 0.60902   , 0.6569195 , ..., 0.4538738 , 0.44272172,
        0.44668537],
       [0.5519114 , 0.5040923 , 0.4953544 , ..., 0.62921256, 0.57721645,
        0.56272763]], dtype=float32)

In [18]:
testPredict

array([[28.707594, 28.595243, 28.546501, ..., 29.245358, 29.14796 ,
        29.000751],
       [31.529638, 31.429403, 31.264673, ..., 30.614187, 30.718132,
        31.004658],
       [28.334492, 28.918531, 29.140392, ..., 27.683336, 28.012436,
        28.235178],
       ...,
       [27.649273, 27.468594, 27.462801, ..., 26.344322, 26.566944,
        27.311634],
       [29.067059, 29.282295, 29.700134, ..., 27.92891 , 27.831627,
        27.866203],
       [28.784119, 28.36698 , 28.290756, ..., 29.458439, 29.004864,
        28.878473]], dtype=float32)

In [19]:
y_test_true

array([[28.663692, 28.663692, 28.744278, ..., 29.630709, 29.630709,
        29.489685],
       [31.645325, 31.423717, 31.343132, ..., 31.947517, 32.08854 ,
        32.551903],
       [28.442085, 29.268078, 29.489685, ..., 28.280916, 29.026325,
        29.106909],
       ...,
       [27.394485, 27.172878, 27.172878, ..., 25.762648, 25.984255,
        26.508055],
       [28.804716, 28.965885, 29.409101, ..., 28.663692, 28.663692,
        28.663692],
       [28.804716, 28.220478, 28.220478, ..., 29.187494, 28.280916,
        28.139893]], dtype=float32)

In [20]:
import numpy as np

def calculate_segment_rmse(predictions, actuals, segment_length=1):
    num_segments = predictions.shape[1] // segment_length
    rmses = []
    
    for i in range(num_segments):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        
        # Extract the segment for both predictions and actuals
        pred_segment = predictions[:, start_idx:end_idx]
        actual_segment = actuals[:, start_idx:end_idx]
        
        # Calculate RMSE for this segment
        segment_rmse = np.sqrt(mean_squared_error(actual_segment.flatten(), pred_segment.flatten()))
        rmses.append(segment_rmse)
    
    return rmses

In [21]:
segment_rmses = calculate_segment_rmse(testPredict, y_test_true)
for i, rmse in enumerate(segment_rmses):
    print(f"RMSE for segment {i + 1}: {rmse:.4f}")

RMSE for segment 1: 0.1668
RMSE for segment 2: 0.2112
RMSE for segment 3: 0.2463
RMSE for segment 4: 0.2817
RMSE for segment 5: 0.3053
RMSE for segment 6: 0.3252
RMSE for segment 7: 0.3376
RMSE for segment 8: 0.3538
RMSE for segment 9: 0.3683
RMSE for segment 10: 0.3720
RMSE for segment 11: 0.3788
RMSE for segment 12: 0.3823
RMSE for segment 13: 0.3825
RMSE for segment 14: 0.3848
RMSE for segment 15: 0.3844
RMSE for segment 16: 0.3860
RMSE for segment 17: 0.3908
RMSE for segment 18: 0.3948
RMSE for segment 19: 0.3987
RMSE for segment 20: 0.4006
RMSE for segment 21: 0.4022
RMSE for segment 22: 0.4060
RMSE for segment 23: 0.4046
RMSE for segment 24: 0.4088
RMSE for segment 25: 0.4178
RMSE for segment 26: 0.4266
RMSE for segment 27: 0.4342
RMSE for segment 28: 0.4465
RMSE for segment 29: 0.4558
RMSE for segment 30: 0.4602
RMSE for segment 31: 0.4665
RMSE for segment 32: 0.4692
RMSE for segment 33: 0.4713
RMSE for segment 34: 0.4755
RMSE for segment 35: 0.4818
RMSE for segment 36: 0.4850
R

In [61]:
model.save("temp_1dcnn.keras")